In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Blood_Type/GSE125791'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular characterization of the microenvironment in CLL-like monoclonal B cell lymphocytosis and early stage chronic lymphocytic leukemia"
!Series_summary	"Several studies have demonstrated an impaired function of the microenvironment in chronic lymphocytic leukemia (CLL), contributing to immune evasion of tumor cells and disease progression. However, in CLL-like monoclonal B cell lymphocytosis (MBL) studies are scarce. Herein, a comprehensive characterization of the microenvironment in 59 MBL, 56 early stage CLL and 31 healthy controls was conducted. Gene expression arrays and qRT-PCR were performed on RNA from CD4+ peripheral blood cells; serum cytokines were measured by immunoassays and proteomic studies; and flow cytometry was applied to evaluate peripheral blood cytotoxic, Th1, exhausted and effector CD4+ T cells, besides monocytic CD14, CD4 and HLA-DR expression. MBL and early stage CLL showed a similar upregulation of cytotoxic and Th1-re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Mock up for background_info just for clarity; this should come from Step 1 output.
background_info = {
    '!Series_title': "Molecular characterization of the microenvironment in CLL-like monoclonal B cell lymphocytosis and early stage chronic lymphocytic leukemia",
    '!Series_summary': "Several studies have demonstrated an impaired function of the microenvironment in chronic lymphocytic leukemia (CLL), contributing to immune evasion of tumor cells and disease progression...",
    '!Series_overall_design': "36 samples: CLL (n=14); CTRL (n=9); MBL (n=13)"
}

# Determine gene expression data availability
if '!Series_title' in background_info and 'Gene expression' in background_info['!Series_summary']:
    is_gene_available = True

# Determine the availability of variables in 'Sample Characteristics Dictionary'
sample_char_dict = {0: ['tissue: Peripheral Blood'], 1: ['type: CLL', 'type: MBL', 'type: CTRL'], 2: ['batch: 1', 'batch: 2', 'batch: 3', 'batch: 4', 'batch: 5', 'batch: 6', 'batch: 9', 'batch: 7', 'batch: 8', 'batch: 10'], 3: ['Sex: m', 'Sex: f'], 4: ['age: 88', 'age: 78', 'age: 72', 'age: 62', 'age: 75', 'age: 73', 'age: 81', 'age: 80', 'age: 76', 'age: 70', 'age: 63', 'age: 83', 'age: 68', 'age: 66', 'age: 60', 'age: 59', 'age: 64', 'age: 82', 'age: 79', 'age: 69', 'age: 67']}

# Assign rows to variables if available and have multiple unique values
if 1 in sample_char_dict and len(set(sample_char_dict[1])) > 1:
    trait_row = 1  # For Blood_Type
if 4 in sample_char_dict and len(set(sample_char_dict[4])) > 1:
    age_row = 4  # For age
if 3 in sample_char_dict and len(set(sample_char_dict[3])) > 1:
    gender_row = 3  # For gender

# Define conversion functions
def convert_trait(value):
    val = value.split(':')[-1].strip()
    return val if val in ['CLL', 'MBL', 'CTRL'] else None

def convert_age(value):
    val = value.split(':')[-1].strip()
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    val = value.split(':')[-1].strip().lower()
    if val == 'm':
        return 1
    elif val == 'f':
        return 0
    else:
        return None

# Save cohort information
save_cohort_info('GSE125791', './preprocessed/Blood_Type/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract and save it
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Blood_Type', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Blood_Type/trait_data/GSE125791.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3581890': ['CLL', 88.0, 1], 'GSM3581891': ['MBL', 78.0, 0], 'GSM3581892': ['MBL', 72.0, 0], 'GSM3581893': ['MBL', 62.0, 0], 'GSM3581894': ['CLL', 75.0, 0], 'GSM3581895': ['CLL', 73.0, 1], 'GSM3581896': ['MBL', 81.0, 0], 'GSM3581897': ['CTRL', 80.0, 0], 'GSM3581898': ['MBL', 76.0, 0], 'GSM3581899': ['CTRL', 75.0, 0], 'GSM3581900': ['MBL', 70.0, 0], 'GSM3581901': ['MBL', 81.0, 1], 'GSM3581902': ['CTRL', 63.0, 1], 'GSM3581903': ['MBL', 83.0, 1], 'GSM3581904': ['CLL', 81.0, 1], 'GSM3581905': ['CTRL', 68.0, 0], 'GSM3581906': ['MBL', 66.0, 0], 'GSM3581907': ['MBL', 60.0, 1], 'GSM3581908': ['CLL', 81.0, 1], 'GSM3581909': ['CLL', 59.0, 1], 'GSM3581910': ['CLL', 75.0, 1], 'GSM3581911': ['CTRL', 75.0, 0], 'GSM3581912': ['CLL', 62.0, 1], 'GSM3581913': ['CLL', 70.0, 1], 'GSM3581914': ['CLL', 63.0, 0], 'GSM3581915': ['CTRL', 64.0, 0], 'GSM3581916': ['CTRL', 60.0, 1], 'GSM3581917': ['CLL', 78.0, 0], 'GSM3581918': ['MBL', 82.0, 1], 'GSM3581919': ['CTRL', 64.0, 0], 'GSM3581920': ['MBL', 79.0, 0],